In [1]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [2]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)

In [3]:
data = open('corpus_hum.txt', encoding="utf-8").read().splitlines()
data_norm = [normalize(text) for text in data]
data_norm = [text for text in data_norm if text]

In [4]:
data_norm[:2]

['абай василий васо иван 1900–2001 русский лингвист родиться 2 15 декабрь 1900 с.коби тифлисский губерния ныне грузия 1925 окончить факультет общественный наука ленинградский университет 1928 аспирантура 1928–1930 сотрудник кавказский историко-археологический институт ан ссср 1930 полвека работать яфетический институт затем институт язык мышление институт языкознание ан ссср ленинград 1950 москва доктор филологический наука 1962 профессор 1969 лауреат государственный премия ссср 1981 почётный член азиатский королевский общество великобритания ирландия 1966 член-корреспондент финно-угорский общество хельсинки 1973 умереть абай москва 18 март 2001',
 'также тема']

In [5]:
len(data_norm)

115082

In [6]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
X = cv.fit_transform(data_norm)

In [7]:
X.shape

(115082, 1000)

## w2v

### w2v самостоятельно обученная

In [8]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [9]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [10]:
data.head(10)

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия
5,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,1,пристав собрать отпечаток палец российский дол...,пристав снять отпечаток палец злостный неплате...
6,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,-1,саратовский дебошир борт самолёт москва хургад...,саратовский дебошир отказываться возвращаться ...
7,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,0,цик хотеть отказаться электронный система подс...,цик отказаться электронный средство подсчёт голос
8,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...,-1,суд петербург оставить дело гибель подросток п...,лондонский гайд-парк это место митинг прежде парк
9,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...,-1,страна опека сократить добыча нефть 1 миллион ...,обама продлить полномочие наса сотрудничество ...


In [11]:
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=300, sg=1)

In [12]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [13]:
dim = 300
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

d:\program files\python\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [14]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [15]:
X_text_w2v.shape

(7227, 600)

In [16]:
from sklearn.model_selection import cross_val_score

In [17]:
y = data['label'].values
print(y.shape)

(7227,)


In [18]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_w2v, y,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.49      0.51      0.50       629
           0       0.47      0.57      0.51       737
           1       0.37      0.23      0.28       441

    accuracy                           0.46      1807
   macro avg       0.44      0.43      0.43      1807
weighted avg       0.45      0.46      0.45      1807



In [19]:
cross_val_score(clf, X_text_w2v, y, scoring='f1_micro', cv=10).mean()

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver 

0.44054273190927323

### w2v rusvectores

In [20]:
data_tag=pd.read_csv('data_paraphraser_norm.csv')

In [21]:
data_tag.head(10)

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться_VERB из_ADP сирия_NOUN россиянин_NOU...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,в_ADP москва_NOUN из_ADP сирия_NOUN вернуться_...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...
5,1,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,пристав_NOUN собирать_VERB отпечаток_NOUN пале...,пристав_NOUN снимать_VERB отпечаток_NOUN палец...
6,-1,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,на_ADP саратовский_ADJ дебошир_NOUN с_ADP борт...,саратовский_ADJ дебошир_NOUN отказываться_VERB...
7,0,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,цик_NOUN хотеть_VERB отказываться_VERB от_ADP ...,цик_NOUN мочь_VERB отказываться_VERB от_ADP эл...
8,-1,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...,суд_NOUN петербург_NOUN оставлять_VERB на_ADP ...,лондонский_ADJ гайд_NOUN парк_NOUN это_PART не...
9,-1,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...,страна_NOUN опек_NOUN сокращать_VERB добыча_NO...,обама_NOUN продлять_VERB полномочие_NOUN нас_N...


In [23]:
y_tag = data_tag['label'].values

In [24]:
rv = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [25]:
rv.most_similar('полицейский_NOUN')

[('милиционер_NOUN', 0.7502387762069702),
 ('полицейский_ADJ', 0.7121201157569885),
 ('полиция_NOUN', 0.7049249410629272),
 ('стражник_NOUN', 0.6457393169403076),
 ('полисмена_NOUN', 0.6368731260299683),
 ('жандарм_NOUN', 0.6188533306121826),
 ('охранник_NOUN', 0.6150203943252563),
 ('бандит_NOUN', 0.6047532558441162),
 ('патрульный_NOUN', 0.6029660105705261),
 ('сыщик_NOUN', 0.601699709892273)]

In [26]:
len(rv['полицейский_NOUN'])

300

In [27]:
dim = 300
X_text_1_rv = np.zeros((len(data_tag['text_1_norm']), dim))
X_text_2_rv = np.zeros((len(data_tag['text_2_norm']), dim))

for i, text in enumerate(data_tag['text_1_norm'].values):
    X_text_1_rv[i] = get_embedding(text, rv, dim)
    
for i, text in enumerate(data_tag['text_2_norm'].values):
    X_text_2_rv[i] = get_embedding(text, rv, dim)

In [28]:
X_text_rv = np.concatenate([X_text_1_rv, X_text_2_rv], axis=1)

In [29]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_rv, y_tag, random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.48      0.51      0.49       629
           0       0.45      0.54      0.49       737
           1       0.34      0.19      0.25       441

    accuracy                           0.45      1807
   macro avg       0.42      0.42      0.41      1807
weighted avg       0.43      0.45      0.43      1807



In [30]:
cross_val_score(clf, X_text_rv, y_tag, scoring='f1_micro', cv=10).mean()

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver 

0.4190825780320484

Самостоятельно обученная модель работает лучше.

## 2 часть

In [52]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, ngram_range=(1, 2), max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.4, max_features=1000,
                min_df=3, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

### SVD

In [53]:
svd = TruncatedSVD(300)

X_text_1_svd = svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd = svd.fit_transform(tfidf.transform(data['text_2_norm']))

X_text_svd = np.concatenate([X_text_1_svd, X_text_2_svd], axis=1)

In [34]:
X_text_svd.shape

(7227, 600)

In [97]:
X_text_svd[:3]

array([[ 0.01465235,  0.03141428,  0.03566327, ..., -0.00704134,
         0.01386935,  0.0031616 ],
       [ 0.00737719,  0.02496352,  0.03580554, ..., -0.01399806,
        -0.02187406,  0.0016654 ],
       [ 0.01180373,  0.04628367,  0.0391773 , ...,  0.02009031,
        -0.04868964,  0.03262126]])

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [98]:
cos_sim_svd = [cosine_similarity(v.reshape(1, -1), X_text_2_svd[i].reshape(1, -1)).tolist()[0] for i, v in enumerate(X_text_1_svd)]
cos_sim_svd = np.array(cos_sim_svd, dtype=np.float64)

In [99]:
cos_sim_svd[:10]

array([[ 0.10696282],
       [ 0.03723929],
       [-0.07655105],
       [ 0.07489418],
       [ 0.00268721],
       [ 0.01349997],
       [-0.06442342],
       [ 0.01353393],
       [-0.01722248],
       [ 0.01650421]])

In [113]:
cos_dis_svd = [cosine_distances(v.reshape(1, -1), X_text_2_svd[i].reshape(1, -1)).tolist()[0] for i, v in enumerate(X_text_1_svd)]
cos_dis_svd = np.array(cos_dis_svd, dtype=np.float64)

In [114]:
cos_dis_svd[:10]

array([[0.89303718],
       [0.96276071],
       [1.07655105],
       [0.92510582],
       [0.99731279],
       [0.98650003],
       [1.06442342],
       [0.98646607],
       [1.01722248],
       [0.98349579]])

#### cross validation

In [58]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_svd,y,random_state=1)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.51      0.54      0.53       629
           0       0.49      0.55      0.52       737
           1       0.38      0.26      0.31       441

    accuracy                           0.48      1807
   macro avg       0.46      0.45      0.45      1807
weighted avg       0.47      0.48      0.47      1807



In [60]:
cross_val_score(clf, X_text_svd, y, scoring='f1_micro', cv=10).mean()

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver 

0.4185398756587693

### NMF

In [66]:
nmf = NMF(300, alpha=0.01)

X_text_1_nmf = nmf.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.fit_transform(tfidf.transform(data['text_2_norm']))

X_text_nmf = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

In [100]:
cos_sim_nmf = [cosine_similarity(v.reshape(1, -1), X_text_2_nmf[i].reshape(1, -1)).tolist()[0] for i, v in enumerate(X_text_1_nmf)]
cos_sim_nmf = np.array(cos_sim_nmf, dtype=np.float64)

In [101]:
cos_sim_nmf[:10]

array([[0.01293521],
       [0.        ],
       [0.        ],
       [0.03965474],
       [0.00214962],
       [0.        ],
       [0.        ],
       [0.0053097 ],
       [0.00342531],
       [0.        ]])

In [115]:
cos_dis_nmf = [cosine_distances(v.reshape(1, -1), X_text_2_nmf[i].reshape(1, -1)).tolist()[0] for i, v in enumerate(X_text_1_nmf)]
cos_dis_nmf = np.array(cos_dis_nmf, dtype=np.float64)
cos_dis_nmf[:10]

array([[0.98706479],
       [1.        ],
       [1.        ],
       [0.96034526],
       [0.99785038],
       [1.        ],
       [1.        ],
       [0.9946903 ],
       [0.99657469],
       [1.        ]])

#### cross validation

In [69]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_nmf, y,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.50      0.54      0.52       629
           0       0.47      0.54      0.51       737
           1       0.34      0.23      0.28       441

    accuracy                           0.46      1807
   macro avg       0.44      0.44      0.43      1807
weighted avg       0.45      0.46      0.45      1807



In [71]:
cross_val_score(clf, X_text_nmf, y, scoring='f1_micro', cv=10).mean()

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver 

0.42615396704931047

### W2V самостоятельно обученная

In [102]:
cos_sim_w2v = [cosine_similarity(v.reshape(1, -1), X_text_2_w2v[i].reshape(1, -1)).tolist()[0] for i, v in enumerate(X_text_1_w2v)]
cos_sim_w2v = np.array(cos_sim_w2v, dtype=np.float64)
cos_sim_w2v[:10]

array([[0.80347232],
       [0.8173343 ],
       [0.88494032],
       [0.72803699],
       [0.84987631],
       [0.8806536 ],
       [0.60933441],
       [0.91536738],
       [0.5287855 ],
       [0.59347246]])

In [116]:
cos_dis_w2v = [cosine_distances(v.reshape(1, -1), X_text_2_w2v[i].reshape(1, -1)).tolist()[0] for i, v in enumerate(X_text_1_w2v)]
cos_dis_w2v = np.array(cos_dis_w2v, dtype=np.float64)
cos_dis_w2v[:10]

array([[0.19652768],
       [0.1826657 ],
       [0.11505968],
       [0.27196301],
       [0.15012369],
       [0.1193464 ],
       [0.39066559],
       [0.08463262],
       [0.4712145 ],
       [0.40652754]])

### W2V rusvectores

In [103]:
cos_sim_rv = [cosine_similarity(v.reshape(1, -1), X_text_2_rv[i].reshape(1, -1)).tolist()[0] for i, v in enumerate(X_text_1_rv)]
cos_sim_rv = np.array(cos_sim_rv, dtype=np.float64)
cos_sim_rv[:10]

array([[0.77144572],
       [0.77986224],
       [0.8482041 ],
       [0.63869703],
       [0.78920513],
       [0.83526636],
       [0.68547967],
       [0.94712941],
       [0.43523004],
       [0.43741337]])

In [137]:
cos_dis_rv = [cosine_distances(v.reshape(1, -1), X_text_2_rv[i].reshape(1, -1)).tolist()[0] for i, v in enumerate(X_text_1_rv)]
cos_dis_rv = np.array(cos_dis_rv, dtype=np.float64)
cos_dis_rv[:10]

array([[0.22855428],
       [0.22013776],
       [0.1517959 ],
       [0.36130297],
       [0.21079487],
       [0.16473364],
       [0.31452033],
       [0.05287059],
       [0.56476996],
       [0.56258663]])

### FastText

In [76]:
fast_text = gensim.models.FastText([text.split() for text in data_norm], size=300, min_n=4, max_n=8)

In [79]:
dim = 300
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

d:\program files\python\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [83]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

In [104]:
cos_sim_ft = [cosine_similarity(v.reshape(1, -1), X_text_2_ft[i].reshape(1, -1)).tolist()[0] for i, v in enumerate(X_text_1_ft)]
cos_sim_ft = np.array(cos_sim_ft, dtype=np.float64)
cos_sim_ft[:10]

array([[0.79974186],
       [0.74884086],
       [0.61154688],
       [0.38153436],
       [0.2737321 ],
       [0.7225023 ],
       [0.43800941],
       [0.88199239],
       [0.23744812],
       [0.13380202]])

In [117]:
cos_dis_ft = [cosine_distances(v.reshape(1, -1), X_text_2_ft[i].reshape(1, -1)).tolist()[0] for i, v in enumerate(X_text_1_ft)]
cos_dis_ft = np.array(cos_dis_ft, dtype=np.float64)
cos_dis_ft[:10]

array([[0.20025814],
       [0.25115914],
       [0.38845312],
       [0.61846564],
       [0.7262679 ],
       [0.2774977 ],
       [0.56199059],
       [0.11800761],
       [0.76255188],
       [0.86619798]])

#### cross validation

In [84]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_ft, y,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.47      0.49      0.48       629
           0       0.46      0.64      0.54       737
           1       0.47      0.14      0.21       441

    accuracy                           0.46      1807
   macro avg       0.47      0.42      0.41      1807
weighted avg       0.47      0.46      0.44      1807



In [85]:
cross_val_score(clf, X_text_ft, y, scoring='f1_micro', cv=10).mean()

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver 

0.44647348485083055

### обучающая выборка из близостей

In [106]:
cos_sim = np.concatenate((cos_sim_svd, cos_sim_nmf, cos_sim_w2v, cos_sim_rv, cos_sim_ft),axis=1)

In [108]:
cos_sim[:3]

array([[ 0.10696282,  0.01293521,  0.80347232,  0.77144572,  0.79974186],
       [ 0.03723929,  0.        ,  0.8173343 ,  0.77986224,  0.74884086],
       [-0.07655105,  0.        ,  0.88494032,  0.8482041 ,  0.61154688]])

In [109]:
train_X, valid_X, train_y, valid_y = train_test_split(cos_sim, y, random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.68      0.62      0.65       629
           0       0.48      0.76      0.59       737
           1       0.75      0.09      0.16       441

    accuracy                           0.55      1807
   macro avg       0.64      0.49      0.47      1807
weighted avg       0.62      0.55      0.51      1807



In [110]:
cross_val_score(clf, cos_sim, y, scoring='f1_micro', cv=10).mean()

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver 

0.5575898225119431

Работает лучше